In [ ]:
# Código 11.0:  Bibliotecas e programas %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#                 rode sempre este código antes dos demais
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression #permite ponderação
# sm.logit usa maxima verossimilhança (tradicional)
# LogisticRegression usa estimação  via solver
import shap
import xgboost as xgb
from sklearn.metrics import roc_curve, auc, precision_recall_curve, auc
from sklearn.metrics import precision_score, recall_score, f1_score,matthews_corrcoef,confusion_matrix, log_loss
import matplotlib.pyplot as plt

In [ ]:
# Código 11.1: Criando o dataframe  kim   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Primeiro baixe os arquivos Kimsport e  Kimsport5 do github
# salve em seu computador
# faça o upload do arquivo no colab
#rode kim=pd.read_excel('/content/sample_data/Kimbal.xlsx') antes de cada algoritmo
#  pois x_train e X_test saõ modificados a cada rodada

In [ ]:
# Código 11.2:REGLOG sem balanceamento
kim=pd.read_excel('/content/sample_data/Kimbal.xlsx')
X= kim.drop('STATUS_MP', axis=1)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
frequenciaoriginal = y.value_counts(dropna=False).sort_index()
print("frequencia amostra original :", frequenciaoriginal)
################################################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento:", frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de teste:", frequencia_test)
#==============================================================================
# rodando a regressao sem Backward selection (preguiça....)
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression( max_iter=1000, random_state=18)
modelo.fit(X_train, y_train)
print("Intercepto:", modelo.intercept_[0])
y_pred_prob = modelo.predict_proba(X_test)[:, 1]
coef = pd.Series(modelo.coef_[0], index=X_train.columns)
print(coef)
#==============================================================================
print("\n\nAvaliação de REGLOG sem balanceamento de amostras")
# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Exibir a AUC
print(f'\nAUC da curva ROC sem balanceamento: {roc_auc:.2f}')

#PRcurve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
# Calculando a AUC da curva Precision-Recall
pr_auc = auc(recall, precision)
print(f"\nAUC da curva Precision-Recall: {pr_auc:.2f}")
#==============================================================================
# Avaliação do modelo com ponto de corte PC
print("\nAvaliação sem balanceamento de amostras")

PC=0.25

y_pred = (y_pred_prob >= PC).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"\nMatriz de Classificação com PC={PC}")
conf_matrix=pd.DataFrame(conf_matrix)
conf_matrix = np.array(conf_matrix)
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real BP', 'Real MP'], columns=['Predito BP', 'Predito MP'])
print(df_conf_matrix)
print(f"\nPrecisão:, {precision:.2f}")
print(f"Recall:, {recall:.2f}")
print(f"F1-score:, {f1:.2f}")
print(f"Mathews correlation coeficient:, {MCC:.2f}")

In [ ]:
#Código 11.3: REG LOG sem balanceamento, mas...ponderando observações
#==============================================================================
# Primeiro faça o upload do arquivo
kim=pd.read_excel('/content/sample_data/Kimbal.xlsx')
X= kim.drop('STATUS_MP', axis=1)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
#==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento:", frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de teste:", frequencia_test)
#==============================================================================
modelo = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=18)
modelo.fit(X_train, y_train)
print("Intercepto:", modelo.intercept_[0])
y_pred_prob = modelo.predict_proba(X_test)[:, 1]
coef = pd.Series(modelo.coef_[0], index=X_train.columns)
print(coef)
#==============================================================================
#Avaliação via AUC

# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Exibir a AUC
print(f'\nAUC da curva ROC sem balanceamento: {roc_auc:.2f}')

#PRcurve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
# Calculando a AUC da curva Precision-Recall
pr_auc = auc(recall, precision)
print(f"\nAUC da curva Precision-Recall: {pr_auc:.2f}")
#==============================================================================
# Avaliação do modelo com ponto de corte PC
print("\nAvaliação sem balanceamento de amostras")
PC=0.6

y_pred = (y_pred_prob >= PC).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"\nMatriz de Classificação com PC={PC}")
conf_matrix=pd.DataFrame(conf_matrix)
conf_matrix = np.array(conf_matrix)
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real BP', 'Real MP'], columns=['Predito BP', 'Predito MP'])
print("\n",df_conf_matrix)
print(f"\nPrecisão:, {precision:.2f}")
print(f"Recall:, {recall:.2f}")
print(f"F1-score:, {f1:.2f}")
print(f"Mathews correlation coeficient:, {MCC:.2f}")

In [ ]:
#Código 11.4:  REGLOG com undersampling método RUS %%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#rodar logística com sm, utilizando BW com p values alfa=0,15
#==============================================================================
# Primeiro faça o upload do arquivo
kim=pd.read_excel('/content/sample_data/Kimbal.xlsx')
X= kim.drop('STATUS_MP', axis=1)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
#==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento:", frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de teste:", frequencia_test)
#==============================================================================
#balanceando com RUS
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_train, y_train = rus.fit_resample(X_train, y_train)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento com RUS:", frequencia_train)
#==============================================================================
X_train = sm.add_constant(X_train)
def backward_elimination(X_train, y_train, significance_level=0.15):
    # Ajustando o modelo de regressão logística
    model = sm.Logit(y_train, X_train)
    result = model.fit(disp=0)
    # Realizando a eliminação BWard
    while max(result.pvalues) > significance_level:
        # Identificar o maior p-valor
        max_p_value = result.pvalues.idxmax()

        # Remover a variável com o maior p-valor
        #print(f"\nRemovendo a variável: {max_p_value} com p-valor: {result.pvalues[max_p_value]}")
        X_train = X_train.drop(columns=[max_p_value])

        # Ajustando o modelo novamente com as variáveis restantes
        model = sm.Logit(y_train, X_train)
        result = model.fit(disp=0)
    return result, X_train
#cuidado: result só existe dentro da função. Nao é valor salvo

# Executando a eliminação BW com o conjunto de treino balanceado
final_model, X_selected = backward_elimination(X_train, y_train)
# O modelo final está em final_model, e as variáveis selecionadas em X_selected
# colocando colunas em X_test para ficar igual a X_selected
selected_columns = X_selected.columns[1:]
# Filter X_test using selected_columns
X_test = X_test[selected_columns]
X_test = sm.add_constant(X_test)  # Add constant back to X_test
#==============================================================================
y_pred_prob = final_model.predict(X_test)

print("\n\nAvaliação com balanceamento com RUS =============================")
# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Exibir a AUC
print(f'\nAUC da curva ROC com RUS: {roc_auc:.2f}')

#PRcurve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
# Calculando a AUC da curva Precision-Recall
pr_auc = auc(recall, precision)
print(f"\nAUC da curva Precision-Recall com RUS: {pr_auc:.2f}")
#==============================================================================
# Avaliação do modelo com ponto de corte

PC=0.7

y_pred = (y_pred_prob >= PC).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\nMatriz de Classificação com PC=",PC)
conf_matrix=pd.DataFrame(conf_matrix)
conf_matrix = np.array(conf_matrix)
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real BP', 'Real MP'], columns=['Predito BP', 'Predito MP'])
print("\n",df_conf_matrix)
print(f"\nPrecisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Mathews correlation coeficient: {MCC:.2f}")

In [ ]:
#Código 11.5: REGLOG com oversampling método SMOTE
#rodar logística
#utilizando BW com p values alfa=0,15
#==============================================================================
# Primeiro faça o upload do arquivo
import pandas as pd
kim=pd.read_excel('/content/sample_data/Kimbal.xlsx')
X= kim.drop('STATUS_MP', axis=1)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
frequenciaoriginal = y.value_counts(dropna=False).sort_index()
print("frequencia amostra original :", frequenciaoriginal)
#==============================================================================

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento:", frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de teste:", frequencia_test)
#==============================================================================
# correção SMOTE para a amostra de treinamento
from imblearn.over_sampling import SMOTE
smo = SMOTE(random_state=18)
X_train, y_train = smo.fit_resample(X_train, y_train)
frequencia_train = y_train.value_counts(dropna=False).sort_index()
print("\nFrequencia amostra treinamento com SMOTE:",frequencia_train)
#==============================================================================
X_train = sm.add_constant(X_train)
def backward_elimination(X_train, y_train, significance_level=0.15):
    # Ajustando o modelo de regressão logística
    model = sm.Logit(y_train, X_train)
    result = model.fit(disp=0)
    # Realizando a eliminação BWard
    while max(result.pvalues) > significance_level:
        # Identificar o maior p-valor
        max_p_value = result.pvalues.idxmax()

        # Remover a variável com o maior p-valor
        #print(f"\nRemovendo a variável: {max_p_value} com p-valor: {result.pvalues[max_p_value]}")
        X_train = X_train.drop(columns=[max_p_value])

        # Ajustando o modelo novamente com as variáveis restantes
        model = sm.Logit(y_train, X_train)
        result = model.fit(disp=0)
    return result, X_train
# Executando a eliminação BW com o conjunto de treino
final_model, X_selected = backward_elimination(X_train, y_train)
# colocando colunas em X_test para ficar igual a X_selected
selected_columns = X_selected.columns[1:]
# Filter X_test using selected_columns
X_test = X_test[selected_columns]
X_test = sm.add_constant(X_test)  # Add constant back to X_test
#==============================================================================
y_pred_prob = final_model.predict(X_test)

print("\n\nAvaliação com balanceamento com SMOTE =============================")
# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Exibir a AUC
print(f'\nAUC da curva ROC com SMOTE: {roc_auc:.2f}')

#PRcurve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
# Calculando a AUC da curva Precision-Recall
pr_auc = auc(recall, precision)
print(f"\nAUC da curva Precision-Recall com SMOTE: {pr_auc:.2f}")
#==============================================================================
# Avaliação do modelo
PC=0.4
y_pred = (y_pred_prob >= PC).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\nMatriz de Classificação com PC=",PC)
conf_matrix=pd.DataFrame(conf_matrix)
conf_matrix = np.array(conf_matrix)
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real BP', 'Real MP'], columns=['Predito BP', 'Predito MP'])
print(df_conf_matrix)
print(f"\nPrecisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Mathews correlation coeficient: {MCC:.2f}")

In [ ]:
# Código 11.6: Rodando a REGLOG com  método SMOTEENN
#rodar logística utilizando BW com p values alfa=0,15
#==============================================================================
# Primeiro faça o upload do arquivo
kim=pd.read_excel('/content/sample_data/Kimbal.xlsx')
X= kim.drop('STATUS_MP', axis=1)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
frequenciaoriginal = y.value_counts(dropna=False).sort_index()
print("frequencia amostra original :", frequenciaoriginal)
#==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)

frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento:", frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de teste:", frequencia_test)
#==============================================================================
# correção SMOTE ENN para a amostra de treinamento
from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=18)
X_train, y_train = sme.fit_resample(X_train, y_train)
frequencia_train = y_train.value_counts(dropna=False).sort_index()
print("\nFrequencia amostra treinamento com SMOTE EEN:",frequencia_train)
#==============================================================================
X_train = sm.add_constant(X_train)
def backward_elimination(X_train, y_train, significance_level=0.15):
    # Ajustando o modelo de regressão logística
    model = sm.Logit(y_train, X_train)
    result = model.fit(disp=0)
    # Realizando a eliminação BWard
    while max(result.pvalues) > significance_level:
        # Identificar o maior p-valor
        max_p_value = result.pvalues.idxmax()

        # Remover a variável com o maior p-valor
        #print(f"\nRemovendo a variável: {max_p_value} com p-valor: {result.pvalues[max_p_value]}")
        X_train = X_train.drop(columns=[max_p_value])

        # Ajustando o modelo novamente com as variáveis restantes
        model = sm.Logit(y_train, X_train)
        result = model.fit(disp=0)
    return result, X_train
# Executando a eliminação BW com o conjunto de treino
final_model, X_selected = backward_elimination(X_train, y_train)
# colocando colunas em X_test para ficar igual a X_selected
selected_columns = X_selected.columns[1:]
# Filter X_test using selected_columns
X_test = X_test[selected_columns]
X_test = sm.add_constant(X_test)  # Add constant back to X_test

#==============================================================================
y_pred_prob = final_model.predict(X_test)

print("\n\nAvaliação com balanceamento com SMOTEEEN =============================")
# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Exibir a AUC
print(f'\nAUC da curva ROC com SMOTE EEN: {roc_auc:.2f}')

#PRcurve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
# Calculando a AUC da curva Precision-Recall
pr_auc = auc(recall, precision)
print(f"\nAUC da curva Precision-Recall com SMOTE EEN: {pr_auc:.2f}")
#==============================================================================
# Avaliação do modelo

PC=0.7
y_pred = (y_pred_prob >= PC).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\nMatriz de Classificação com PC=",PC)
conf_matrix=pd.DataFrame(conf_matrix)
conf_matrix = np.array(conf_matrix)
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real BP', 'Real MP'], columns=['Predito BP', 'Predito MP'])
print("\n",df_conf_matrix)
print(f"\nPrecisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Mathews correlation coeficient: {MCC:.2f}")

In [ ]:
#Código 11.7: Rodando com Random Forest sem balancear
#==============================================================================
# Primeiro faça o upload do arquivo
import pandas as pd
kim=pd.read_excel('/content/sample_data/Kimbal.xlsx')
X= kim.drop('STATUS_MP', axis=1)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
frequenciaoriginal = y.value_counts(dropna=False).sort_index()
print("frequencia amostra original :", frequenciaoriginal)
#==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento:", frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de teste:", frequencia_test)
#==============================================================================
## utilizando RF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
#Se você não especificar bootstrap no seu modelo de Random Forest, o valor padrão será True.
rf = RandomForestClassifier(random_state=18,
                            min_samples_split=100,
                            min_samples_leaf=50,
                            bootstrap=True,
                            oob_score=True)  # Usar amostras OOB para validação
param_grid = {
    'n_estimators': [100, 200], # número de rodadas
    'max_depth': [10, 15], # profundidade da árvore / qto maior, maior o risco de overfitting
    'max_features': ['sqrt', 'log2'] #quantidade de vars a serem selecionadas
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1, verbose=2)
#n_jobs=-1: Usará todos os núcleos de CPU disponíveis no seu computador para executar a busca
#o critério de avaliação é especificado pelo parâmetro scoring. Eu prefiro AUROC
grid_search.fit(X_train, y_train)

# Mostrar os resultados de AUC para cada combinação de parâmetros
# grid_search.cv_results_ contém os resultados de todas as combinações
results = grid_search.cv_results_
# Exibir os valores de AUC para cada combinação de hiperparâmetros
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"AUC: {mean_score:.4f} | Hiperparâmetros: {params}")

print("\n")
print("Melhores parâmetros encontrados:", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_

y_pred_prob = best_rf_model.predict_proba(X_test)[:, 1] # probMP que é class=1
#==============================================================================
print("\n\nAvaliação com balanceamento com RF sem balancear ===================")
# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Exibir a AUC
print(f'\nAUC da curva ROC com RF sem balancear: {roc_auc:.2f}')

#PRcurve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
# Calculando a AUC da curva Precision-Recall
pr_auc = auc(recall, precision)
print(f"\nAUC da curva Precision-Recall com RF sem balancear: {pr_auc:.2f}")
#==============================================================================

# Avaliação do modelo

PC=0.2
y_pred = (y_pred_prob >= PC).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\nMatriz de Classificação com PC=",PC)
conf_matrix=pd.DataFrame(conf_matrix)
conf_matrix = np.array(conf_matrix)
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real BP', 'Real MP'], columns=['Predito BP', 'Predito MP'])
print(df_conf_matrix)
print(f"\nPrecisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Mathews correlation coeficient: {MCC:.2f}")

In [ ]:
#Código 11.8: Rodando com BalancedRandomForestClassifier
#==============================================================================
# Primeiro faça o upload do arquivo
import pandas as pd
kim=pd.read_excel('/content/sample_data/Kimbal.xlsx')
X= kim.drop('STATUS_MP', axis=1)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
frequenciaoriginal = y.value_counts(dropna=False).sort_index()
print("frequencia amostra original :", frequenciaoriginal)
#==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento:", frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de teste:", frequencia_test)
#==============================================================================
# correção BalancedRandomForestClassifier para a amostra de treinamento
from imblearn.ensemble import BalancedRandomForestClassifier
clf = BalancedRandomForestClassifier(
    sampling_strategy="majority", replacement=True, max_depth=3, random_state=11,
    bootstrap=True)
clf.fit(X_train, y_train)
# Prevendo
y_pred_prob = clf.predict_proba(X_test)[:, 1]
# Avaliação
frequencia_train = y_train.value_counts(dropna=False).sort_index()
print("\nFrequencia amostra treinamento com BalancedRandomForestClassifier:",frequencia_train)
#==============================================================================
print("\n\nAvaliação com balanceamento com BalancedRandomForestClassifier ========")
# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Exibir a AUC
print(f'\nAUC da curva ROC com BalancedRandomForestClassifier: {roc_auc:.2f}')

#PRcurve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
# Calculando a AUC da curva Precision-Recall
pr_auc = auc(recall, precision)
print(f"\nAUC da curva Precision-Recall com BalancedRandomForestClassifier: {pr_auc:.2f}")
#==============================================================================
# Avaliação do modelo

PC=0.55

y_pred = (y_pred_prob >= PC).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\nMatriz de Classificação com PC=",PC)
conf_matrix=pd.DataFrame(conf_matrix)
conf_matrix = np.array(conf_matrix)
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real BP', 'Real MP'], columns=['Predito BP', 'Predito MP'])
print(df_conf_matrix)
print(f"\nPrecisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Mathews correlation coeficient: {MCC:.2f}")

In [ ]:
#Código 11.9: Rodando com XGBoost sem balancear
#rodar logística sm. utilizando BW com p values alfa=0,15
#==============================================================================
# Primeiro faça o upload do arquivo
import pandas as pd
kim=pd.read_excel('/content/sample_data/Kimbal.xlsx')
X= kim.drop('STATUS_MP', axis=1)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
frequenciaoriginal = y.value_counts(dropna=False).sort_index()
print("frequencia amostra original :", frequenciaoriginal)
#==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento:", frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de teste:", frequencia_test)
#==============================================================================
## utilizabndo xgb

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4,7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [10,20]}
model = xgb.XGBClassifier(objective='binary:logistic',
                          eval_metric='logloss',early_stopping_rounds=10,learning_rate=0.1,
                          use_label_encoder=False)
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_log_loss',
    cv=3,  # Cross-validation com 3 folds
    verbose=1,
    n_jobs=-1)

grid_search.fit(X_train, y_train,  eval_set=[(X_test, y_test)], verbose=False)

best_model = grid_search.best_estimator_

# Para acessar o número de iterações antes de ocorrer o early stop:
aux=best_model.get_booster().best_iteration
print(f'Número de árvores necessarias (early stop):  {aux}')

# Predição
y_pred_prob = best_model.predict_proba(X_test)[:, 1]
#==============================================================================
print("\n\nAvaliação com balanceamento com XGBoost =============================")
# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Exibir a AUC
print(f'\nAUC da curva ROC com XGBoost sem balancear: {roc_auc:.2f}')

#PRcurve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
# Calculando a AUC da curva Precision-Recall
pr_auc = auc(recall, precision)
print(f"\nAUC da curva Precision-Recall com XGBoost sem balancear: {pr_auc:.2f}")
#==============================================================================
# Avaliação do modelo

PC=0.2
y_pred = (y_pred_prob >= PC).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\nMatriz de Classificação com PC=",PC)
conf_matrix=pd.DataFrame(conf_matrix)
conf_matrix = np.array(conf_matrix)
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real BP', 'Real MP'], columns=['Predito BP', 'Predito MP'])
print(df_conf_matrix)
print(f"\nPrecisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Mathews correlation coeficient: {MCC:.2f}")

In [ ]:
#Código 11.10: rodando com XGBoost balanceando com RUS
#==============================================================================
# Primeiro faça o upload do arquivo
import pandas as pd
kim=pd.read_excel('/content/sample_data/Kimbal.xlsx')
X= kim.drop('STATUS_MP', axis=1)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
frequenciaoriginal = y.value_counts(dropna=False).sort_index()
print("frequencia amostra original :", frequenciaoriginal)
#==============================================================================

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento:", frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de teste:", frequencia_test)

#==============================================================================
#balanceando com RUS
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_train, y_train = rus.fit_resample(X_train, y_train)
frequencia_train = y_train['STATUS_MP'].value_counts(dropna=False).sort_index()
print("\nFrequencia amostra de treinamento com RUS:", frequencia_train)
#==============================================================================
## utilizabndo xgb

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4,7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [10,20]}
model = xgb.XGBClassifier(objective='binary:logistic',
                          eval_metric='logloss',early_stopping_rounds=10,learning_rate=0.1,
                          use_label_encoder=False)
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_log_loss',
    cv=3,  # Cross-validation com 3 folds
    verbose=1,
    n_jobs=-1)

grid_search.fit(X_train, y_train,  eval_set=[(X_test, y_test)], verbose=False)

best_model = grid_search.best_estimator_

# Para acessar o número de iterações antes de ocorrer o early stop:
aux=best_model.get_booster().best_iteration
print(f'Número de árvores necessarias (early stop):  {aux}')

# Predição e cálculo do LogLoss
y_pred_prob = best_model.predict_proba(X_test)[:, 1]
#==============================================================================
print("\n\nAvaliação com balanceamento com XGBoost =============================")
# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Exibir a AUC
print(f'\nAUC da curva ROC com XGBoost sem balancear: {roc_auc:.2f}')

#PRcurve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
# Calculando a AUC da curva Precision-Recall
pr_auc = auc(recall, precision)
print(f"\nAUC da curva Precision-Recall com XGBoost sem balancear: {pr_auc:.2f}")
#==============================================================================

# Avaliação do modelo

PC=0.6
y_pred = (y_pred_prob >= PC).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\nMatriz de Classificação com PC=",PC)
conf_matrix=pd.DataFrame(conf_matrix)
conf_matrix = np.array(conf_matrix)
df_conf_matrix = pd.DataFrame(conf_matrix, index=['Real BP', 'Real MP'], columns=['Predito BP', 'Predito MP'])
print(df_conf_matrix)
print(f"\nPrecisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Mathews correlation coeficient: {MCC:.2f}")